### Yelp Review Classification (Positive vs Negative)

Data Source: https://www.kaggle.com/c/yelp-recruiting

In [44]:
from __future__ import print_function
import pandas as pd
import numpy as np

In [20]:
# read data to dataframe
fname = '../yelp_training_set/yelp_training_set_review.json'
data = pd.read_json(fname, lines=True)
print(data.shape)
print(data.head())
print()

(229907, 8)
              business_id       date               review_id  stars  \
0  9yKzy9PApeiPPOUJEtnvkg 2011-01-26  fWKvX83p0-ka4JS3dc6E5A      5   
1  ZRJwVLyzEJq1VAihDhYiow 2011-07-27  IjZ33sJrzXqU-0X6U8NwyA      5   
2  6oRAC4uyJCsJl1X0WZpVSA 2012-06-14  IESLBzqUCLdSzSqm0eCSxQ      4   
3  _1QQZuf4zZOyFCvXc0o6Vg 2010-05-27  G-WvGaISbqqaMHlNnByodA      5   
4  6ozycU1RpktNG2-1BroVtw 2012-01-05  1uJFq2r5QfJG_6ExMRCaGw      5   

                                                text    type  \
0  My wife took me here on my birthday for breakf...  review   
1  I have no idea why some people give bad review...  review   
2  love the gyro plate. Rice is so good and I als...  review   
3  Rosie, Dakota, and I LOVE Chaparral Dog Park!!...  review   
4  General Manager Scott Petello is a good egg!!!...  review   

                  user_id                                    votes  
0  rLtl8ZkDX5vH5nAx9C3q5Q  {u'funny': 0, u'useful': 5, u'cool': 2}  
1  0a2KyEL0d3Yb1V6aivbIuQ  {u'funny': 

In [15]:
# check information
# print('Type contains:')
# print(data['type'].unique())
# print('Null values:')
# print(data.isnull().any())
# print(data['stars'].unique())
'''
results show:
every line is a review entry;
no Null values present
'''

[5 4 2 3 1]


'\nresults show:\nevery line is a review entry;\nno Null values present\n'

In [79]:
# drop features
# since we are only classifying positive and negative reviews, take only the useful features
useful_features = ['stars', 'text']
data_useful = data.loc[:, useful_features]
data_useful = data_useful.loc[data['stars'] != 3]

# convert stars to binary values 0 and 1
data_useful['label'] = data_useful['stars'].apply(lambda x : 1 if x > 3 else 0)
data_useful.drop(['stars'], axis=1, inplace=True)
print(data_useful.head())
print()

# select a subset with equal number of positive and negative reviews
data_pos = data_useful.loc[data_useful['label'] == 1]
data_neg = data_useful.loc[data_useful['label'] == 0]
num_perclass = data_neg.shape[0]
data_pos_spl = data_pos.sample(n=num_perclass, random_state=100)
data_spl = pd.concat([data_pos_spl, data_neg], ignore_index=True)
data_spl = data_spl.reindex(np.random.permutation(data_spl.index))
print(data_spl.head())
print(data_spl['label'].value_counts())
print()

# separte x and y data
X_data = data_spl['text']
Y_data = data_spl['label']
print('X shape:', X_data.shape)
print('Y shape:', Y_data.shape)
print()

# check number of positive and negative reviews
print(Y_data.value_counts())

                                                text  label
0  My wife took me here on my birthday for breakf...      1
1  I have no idea why some people give bad review...      1
2  love the gyro plate. Rice is so good and I als...      1
3  Rosie, Dakota, and I LOVE Chaparral Dog Park!!...      1
4  General Manager Scott Petello is a good egg!!!...      1

                                                    text  label
13175  Started coming here the first week it opened, ...      1
75390  Thumbs mostly down for me.  Something importan...      0
33307  I looooooove ATS! The bean burritos are the be...      1
10249  I stopped by for the first time this past week...      1
10709  I like to find things to do that just sound fu...      1
1    38473
0    38473
Name: label, dtype: int64

X shape: (76946L,)
Y shape: (76946L,)

1    38473
0    38473
Name: label, dtype: int64


In [80]:
# partition train and test sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_data, Y_data, random_state=0)
print('Train shape:', X_train.shape)
print(sum(y_train == 1))
print('Test shape:', X_test.shape)

Train shape: (57709L,)
28810
Test shape: (19237L,)


### Bag of Words Method

Use frequency vector of word occurences. It does not keep the order information.
Tokenizing and stop-words filtering are done internally.

CountVectorizer simply computes the frequency of words. TfidfVectorizaer considers the relative frequency - it pays more attention to the words that have high frequency in certain documents and not all.

In [81]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [72]:
# TESTING
# text = pd.Series(['is are Main main, clothes, cloth, mice, mouse running, run, meets, met, meet'])
# print(text)
# vect = CountVectorizer()
# vect.fit(text)
# feat_names = vect.get_feature_names()
# print(feat_names)
'''the tokenization in sklearn simply extract words (ignoring letter case)'''

'the tokenization in sklearn simply extract words (ignoring letter case)'

#### use CountVectorizer

In [30]:
vect = CountVectorizer()

In [61]:
vect = CountVectorizer(min_df=3)

#### use TF-IDF

In [56]:
vect = TfidfVectorizer()

In [66]:
vect = TfidfVectorizer(min_df=3)

In [82]:
vect = TfidfVectorizer(ngram_range=(1,2))

### Transform data

In [83]:
vect.fit(X_train)
feat_names = vect.get_feature_names()
print('num_features:', len(feat_names))
# print(feat_names[::1000])

X_train_vect = vect.transform(X_train)
print(X_train_vect.shape)

X_test_vect = vect.transform(X_test)
print(X_test_vect.shape)

num_features: 1454125
(57709, 1454125)
(19237, 1454125)


### use model

In [84]:
# use logistic regression
clf = LogisticRegression()
clf.fit(X_train_vect, y_train)
preds = clf.predict(X_test_vect)

In [85]:
# compute accuracy
accuracy = accuracy_score(y_test, preds)
print(accuracy)

0.929926703748


In [86]:
feat_names = np.array(feat_names)
sorted_coef_index = clf.coef_[0].argsort()

print('Smallest Coefs:\n{}\n'.format(feat_names[sorted_coef_index[:10]]))
print('Largest Coefs: \n{}'.format(feat_names[sorted_coef_index[:-11:-1]]))

Smallest Coefs:
[u'not' u'no' u'bland' u'worst' u'ok' u'bad' u'terrible' u'nothing'
 u'mediocre' u'rude']

Largest Coefs: 
[u'great' u'love' u'delicious' u'awesome' u'amazing' u'excellent' u'best'
 u'good' u'perfect' u'definitely']


### References
- https://machinelearningmastery.com/prepare-text-data-machine-learning-scikit-learn/
- http://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html
- http://scikit-learn.org/stable/modules/feature_extraction.html